# 简单 RAG 中的上下文块标题（CCH）

检索增强生成（RAG）通过在生成响应之前检索相关的外部知识，提高了语言模型的事实准确性。然而，标准的块分割常常丢失重要的上下文，使得检索效果大打折扣。

上下文块标题（CCH）通过在嵌入之前为每个块添加高级上下文（如文档标题或节标题）来增强 RAG。这提高了检索质量，并防止生成脱离上下文的响应。

## 本笔记本中的步骤：

1. **数据摄取**：加载和预处理文本数据。
2. **带有上下文标题的分块**：提取节标题并将其添加到块的前面。
3. **嵌入创建**：将带有上下文增强的块转换为数值表示。
4. **语义搜索**：根据用户查询检索相关块。
5. **响应生成**：使用语言模型根据检索到的文本生成响应。
6. **评估**：使用评分系统评估响应的准确性。

## 环境设置

In [2]:
# fitz库需要从pymudf那里安装
%pip install --quiet --force-reinstall pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 60.9 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
import json
from openai import OpenAI
import fitz
from tqdm import tqdm

## 提取文本并识别节标题
我们从 PDF 中提取文本，同时识别节标题（潜在的块标题）。

In [4]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file.

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    # Open the PDF file
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Initialize an empty string to store the extracted text

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # Get the page
        text = page.get_text("text")  # Extract text from the page
        all_text += text  # Append the extracted text to the all_text string

    return all_text  # Return the extracted text

## OpenAI API Client

In [5]:
# colab环境
from google.colab import userdata
# 使用火山引擎
api_key = userdata.get("ARK_API_KEY")
base_url = userdata.get("ARK_BASE_URL")

In [6]:
# Initialize the OpenAI client with the base URL and API key
client = OpenAI(
    base_url=base_url,
    api_key=api_key
)

## 带有上下文标题的文本分块
为了提高检索效果，我们使用大语言模型为每个块生成描述性标题。

In [7]:
def generate_chunk_header(chunk, model="doubao-lite-128k-240828"):
    """
    使用大语言模型为给定的文本块生成标题。.

    Args:
    chunk (str): 要生成标题的文本块.
    model (str): LLM. 默认为 "doubao-lite-128k-240828".

    Returns:
    str: 生成的标题.
    """
    # Define the system prompt to guide the AI's behavior
    system_prompt = "Generate a concise and informative title for the given text."

    # Generate a response from the AI model based on the system prompt and text chunk
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": chunk}
        ]
    )

    # Return the generated header/title, stripping any leading/trailing whitespace
    return response.choices[0].message.content.strip()

In [8]:
def chunk_text_with_headers(text, n, overlap):
    """
    分块并生成标题

    Args:
    text (str): The full text to be chunked.
    n (int): The chunk size in characters.
    overlap (int): Overlapping characters between chunks.

    Returns:
    List[dict]: A list of dictionaries with 'header' and 'text' keys.
    """
    chunks = []  # Initialize an empty list to store chunks

    # Iterate through the text with the specified chunk size and overlap
    for i in range(0, len(text), n - overlap):
        chunk = text[i:i + n]  # Extract a chunk of text
        header = generate_chunk_header(chunk)  # Generate a header for the chunk using LLM
        chunks.append({"header": header, "text": chunk})  # Append the header and chunk to the list

    return chunks  # Return the list of chunks with headers

## 从 PDF 文件中提取和分块文本
现在，我们加载 PDF 文件，提取文本并将其分割成块。

In [9]:
# Define the PDF file path
pdf_path = "./data/AI_Information.pdf"

# Extract text from the PDF file
extracted_text = extract_text_from_pdf(pdf_path)

# Chunk the extracted text with headers
# We use a chunk size of 1000 characters and an overlap of 200 characters
text_chunks = chunk_text_with_headers(extracted_text, 1000, 200)

# Print a sample chunk with its generated header
print("Sample Chunk:")
print("Header:", text_chunks[0]['header'])
print("Content:", text_chunks[0]['text'])

Sample Chunk:
Header: "Understanding Artificial Intelligence: Its History, Concepts & Advances"
Content: Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the development and deployment of AI. 
Historical Context 
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop 
in 1956 is widely considered the birthpla

## Creating Embeddings for Headers and Text
We create embeddings for both headers and text to improve retrieval accuracy.

In [10]:
def create_embeddings(text, model="doubao-embedding-text-240715"):
    """
    Creates embeddings for the given text.

    Args:
    text (str): The input text to be embedded.
    model (str): The embedding model to be used. Default is "doubao-embedding-text-240715".

    Returns:
    np.ndarray: The response containing the embedding for the input text.
    """
    # Create embeddings using the specified model and input text
    response = client.embeddings.create(
        model=model,
        input=text
    )
    # Return the embedding from the response
    return response.data[0].embedding

In [11]:
# Generate embeddings for each chunk
embeddings = []  # Initialize an empty list to store embeddings

# Iterate through each text chunk with a progress bar
for chunk in tqdm(text_chunks, desc="Generating embeddings"):
    # Create an embedding for the chunk's text
    text_embedding = create_embeddings(chunk["text"])
    # Create an embedding for the chunk's header
    header_embedding = create_embeddings(chunk["header"])
    # Append the chunk's header, text, and their embeddings to the list
    embeddings.append({"header": chunk["header"], "text": chunk["text"], "embedding": text_embedding, "header_embedding": header_embedding})

Generating embeddings: 100%|██████████| 42/42 [00:48<00:00,  1.16s/it]


## 执行语义搜索
我们实现余弦相似度，以找到与用户查询最相关的文本块。

In [12]:
def cosine_similarity(vec1, vec2):
    """
    Computes cosine similarity between two vectors.

    Args:
    vec1 (np.ndarray): First vector.
    vec2 (np.ndarray): Second vector.

    Returns:
    float: Cosine similarity score.
    """
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [13]:
def semantic_search(query, chunks, k=5):
    """
    Searches for the most relevant chunks based on a query.

    Args:
    query (str): User query.
    chunks (List[dict]): List of text chunks with embeddings.
    k (int): Number of top results.

    Returns:
    List[dict]: Top-k most relevant chunks.
    """
    # Create an embedding for the query
    query_embedding = create_embeddings(query)

    similarities = []  # Initialize a list to store similarity scores

    # Iterate through each chunk to calculate similarity scores
    for chunk in chunks:
        # 计算查询与文本块的余弦相似度
        sim_text = cosine_similarity(np.array(query_embedding), np.array(chunk["embedding"]))
        # 计算查询与标题的余弦相似度
        sim_header = cosine_similarity(np.array(query_embedding), np.array(chunk["header_embedding"]))
        # 计算平均相似度
        avg_similarity = (sim_text + sim_header) / 2

        similarities.append((chunk, avg_similarity))

    # Sort the chunks based on similarity scores in descending order
    similarities.sort(key=lambda x: x[1], reverse=True)
    # Return the top-k most relevant chunks
    return [x[0] for x in similarities[:k]]

## 在提取的块上运行查询

In [14]:
# Load validation data
with open('./data/val.json') as f:
    data = json.load(f)

query = data[0]['question']

# Retrieve the top 2 most relevant text chunks
top_chunks = semantic_search(query, embeddings, k=2)

# Print the results
print("Query:", query)
for i, chunk in enumerate(top_chunks):
    print(f"Header {i+1}: {chunk['header']}")
    print(f"Content:\n{chunk['text']}\n")

Query: What is 'Explainable AI' and why is it considered important?
Header 1: "Key Aspects of Building Trust in AI: Transparency, Explainability, etc."
Content:
systems. Explainable AI (XAI) 
techniques aim to make AI decisions more understandable, enabling users to assess their 
fairness and accuracy. 
Privacy and Data Protection 
AI systems often rely on large amounts of data, raising concerns about privacy and data 
protection. Ensuring responsible data handling, implementing privacy-preserving techniques, 
and complying with data protection regulations are crucial. 
Accountability and Responsibility 
Establishing accountability and responsibility for AI systems is essential for addressing potential 
harms and ensuring ethical behavior. This includes defining roles and responsibilities for 
developers, deployers, and users of AI systems. 
Chapter 20: Building Trust in AI 
Transparency and Explainability 
Transparency and explainability are key to building trust in AI. Making AI syst

## 基于检索到的块生成响应

In [15]:
# Define the system prompt for the AI assistant
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

def generate_response(system_prompt, user_message, model="doubao-lite-128k-240828"):
    """
    Generates a response from the AI model based on the system prompt and user message.

    Args:
    system_prompt (str): The system prompt to guide the AI's behavior.
    user_message (str): The user's message or query.
    model (str): The model to be used for generating the response. Default is "doubao-lite-128k-240828".

    Returns:
    dict: The response from the AI model.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# Create the user prompt based on the top chunks
user_prompt = "\n".join([f"Header: {chunk['header']}\nContent:\n{chunk['text']}" for chunk in top_chunks])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# Generate AI response
ai_response = generate_response(system_prompt, user_prompt)

## 评估 AI 响应
我们将 AI 响应与预期答案进行比较，并分配一个分数。

In [16]:
# Define evaluation system prompt
evaluate_system_prompt = """You are an intelligent evaluation system.
Assess the AI assistant's response based on the provided context.
- Assign a score of 1 if the response is very close to the true answer.
- Assign a score of 0.5 if the response is partially correct.
- Assign a score of 0 if the response is incorrect.
Return only the score (0, 0.5, or 1)."""

# Extract the ground truth answer from validation data
true_answer = data[0]['ideal_answer']

# Construct evaluation prompt
evaluation_prompt = f"""
User Query: {query}
AI Response: {ai_response}
True Answer: {true_answer}
{evaluate_system_prompt}
"""

# Generate evaluation score
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# Print the evaluation score
print("Evaluation Score:", evaluation_response.choices[0].message.content)

Evaluation Score: 1
